In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Generating Plots for Final Report

- Llama-3.1-70b  Recommendations vs. Content-Based Filtering
- Content-Based Filtering vs Collaborative Filtering
- Collaborative-Based Filtering vs Hybrid Filtering


For each comparison mentioned above, include charts for Hit Rate, Mean Average Precision, and Normalized Discounted Cumulative Gain charts*

In [201]:
def create_bar_chart_plotly(df, x, y, color, title, labels, category_orders=None, subplot_titles=None, yaxis_range=None):
    df[x] = df[x].astype(str)

    if isinstance(y, list):
        fig = make_subplots(1, len(y), subplot_titles=subplot_titles)
        
        for idx, metric in enumerate(y):
            temp_fig = px.bar(
                df,
                x=x,
                y=metric,
                color=color,
                barmode="group",
                # title=title,
                labels=labels,
                category_orders=category_orders,
            )
            for trace in temp_fig.data:
                trace.legendgroup = trace.name
                trace.showlegend = True if idx==0 else False
                fig.append_trace(
                    trace,
                    row=1, col=idx+1,
                )
        
        fig.update_xaxes(tickvals=sorted(df[x].unique()))
        fig.update_layout(
            title_text=title,
            legend=dict(orientation='h', yanchor='bottom', xanchor='left', y=-0.3)
            # showlegend=False
        )
    else:
        fig = px.bar(
            df,
            x=x,
            y=y,
            color=color,
            barmode="group",
            title=title,
            labels=labels,
            category_orders=category_orders
        )

        fig.update_xaxes(tickvals=sorted(df[x].unique()))
        fig.update_layout(showlegend=False)
    
    if yaxis_range:
        # Using update_layout()
        fig.update_layout(yaxis_range=yaxis_range)

    fig.show()

### Vanilla LLM Recommendations vs. Content-Based Filtering

In [187]:
df = pd.read_csv("offline_eval_results.csv")
llm_df = pd.read_csv("llm_offline_eval_results.csv")

df = pd.concat([df, llm_df]).reset_index(drop=True)

df = df.sort_values(by=['k', 'name'], ascending=[True, True])

# dropping Image Summary (Tone) rows
df = df[df['name']!="Image Summary (Tone)"]

df.head(5)


,name,k,hit_rate,mean_avg_prec,ndcg
8,Image Summary (Keywords),1,0.180213,0.180213,0.0
20,Llama-3.1 Recommendations,1,0.277850,0.277850,0.0
0,Normal,1,0.165152,0.165152,0.0
12,Screenshot Summary (Keywords),1,0.185926,0.185926,0.0
16,Screenshot and Header Image Keywords,1,0.203064,0.203064,0.0


In [195]:
create_bar_chart_plotly(
    df, 
    'k', 
    ['hit_rate', 'mean_avg_prec', 'ndcg'], 
    'name', 
    title="Llama-3.1-70b vs. Content-based Recommenders", 
    labels={"k": "k", "hit_rate": "Hit Rate", "name": "Method"},
    category_orders={
        'name': [
            'Llama-3.1 Recommendations',
            'Normal',
            'Image Summary (Keywords)',
            'Screenshot Summary (Keywords)',
            'Screenshot and Header Image Keywords',
        ]
    },
    subplot_titles=['HitRate@k', 'MAP@k', 'NDCG@k']
)

### Content-Based Filtering vs Collaborative Filtering

In [189]:
collab_filtering_df = pd.read_csv("collaborative_offline_eval_results.csv")
collab_filtering_df

# combine results from collaborative filtering with results from content-based filtering
cbf_df = pd.read_csv("offline_eval_results.csv")
cbf_df = cbf_df[cbf_df['name']=='Screenshot and Header Image Keywords']

cf_cbf_df = pd.concat([collab_filtering_df, cbf_df])
cf_cbf_df

,name,k,hit_rate,mean_avg_prec,ndcg
0,Memory-Based Collaborative Filtering,1,0.641392,0.641392,0.000000
1,Memory-Based Collaborative Filtering,5,0.896650,0.490548,0.767332
2,Memory-Based Collaborative Filtering,10,0.952480,0.407150,0.763806
3,Memory-Based Collaborative Filtering,20,0.977668,0.345812,0.744820
16,Screenshot and Header Image Keywords,1,0.203064,0.203064,0.000000
17,Screenshot and Header Image Keywords,5,0.502207,0.155596,0.375539
18,Screenshot and Header Image Keywords,10,0.654375,0.131135,0.422058
19,Screenshot and Header Image Keywords,20,0.776681,0.106037,0.443664


In [196]:
create_bar_chart_plotly(
    cf_cbf_df, 
    'k', 
    ['hit_rate', 'mean_avg_prec', 'ndcg'], 
    'name', 
    title="Enhanced Content-Based Recommender vs. Collaborative Recommender", 
    labels={"k": "k", "hit_rate": "Hit Rate", "name": "Method"},
    category_orders={
        'name': [
            'Memory-Based Collaborative Filtering',
            'Screenshot and Header Image Keywords',
        ]
    },
    subplot_titles=['HitRate@k', 'MAP@k', 'NDCG@k']
)

### Collaborative-Based Filtering vs Hybrid Filtering

In [198]:
# hybrid_filtering_df = pd.read_csv("hybrid_hyperparameter_tuning_20250729_014354.csv")
# hybrid_filtering_df_v2 = pd.read_csv("hybrid_hyperparameter_tuning_20250729_155537.csv")
# hybrid_filtering_df = pd.concat([hybrid_filtering_df, hybrid_filtering_df_v2])

# def get_method_name(row):
#     return f"Weight={row['weight']} | Num_Train_Examples={row['num_train_examples']}"
# hybrid_filtering_df['method'] = hybrid_filtering_df[['weight', 'num_train_examples']].apply(get_method_name, axis=1)

# temp_hybrid_filtering_df = hybrid_filtering_df.rename(columns={'method': 'name'})
# temp_hybrid_filtering_df = temp_hybrid_filtering_df[['name', 'k', 'hit_rate', 'mean_avg_prec', 'ndcg']]
# temp_hybrid_filtering_df = temp_hybrid_filtering_df[temp_hybrid_filtering_df['name']=="Weight=0.999 | Num_Train_Examples=nan"]

temp_hybrid_filtering_df = pd.read_csv("switching_hybrid_model_results.csv")
temp_hybrid_filtering_df['name'] = "Switching Hyrbid Model"

combined_df = pd.concat([collab_filtering_df, temp_hybrid_filtering_df])
combined_df

,name,k,hit_rate,mean_avg_prec,ndcg,game_threshold,num_train_examples,mrr
0,Memory-Based Collaborative Filtering,1,0.641392,0.641392,0.000000,NaN,NaN,NaN
1,Memory-Based Collaborative Filtering,5,0.896650,0.490548,0.767332,NaN,NaN,NaN
2,Memory-Based Collaborative Filtering,10,0.952480,0.407150,0.763806,NaN,NaN,NaN
3,Memory-Based Collaborative Filtering,20,0.977668,0.345812,0.744820,NaN,NaN,NaN
0,Switching Hyrbid Model,1,0.641392,0.641392,0.000000,5.0,NaN,0.641392
1,Switching Hyrbid Model,5,0.896650,0.490574,0.767351,5.0,NaN,0.742708
2,Switching Hyrbid Model,10,0.952480,0.407141,0.763806,5.0,NaN,0.750434
3,Switching Hyrbid Model,20,0.977668,0.345803,0.744820,5.0,NaN,0.752290


In [199]:
create_bar_chart_plotly(
    combined_df, 
    'k', 
    ['hit_rate', 'mean_avg_prec', 'ndcg'], 
    'name', 
    title="Collaborative Recommender vs Hybrid Recommender", 
    labels={"k": "k", "hit_rate": "Hit Rate", "name": "Method"},
    subplot_titles=['HitRate@k', 'MAP@k', 'NDCG@k']
)

In [205]:
create_bar_chart_plotly(
    combined_df[combined_df['k']=='5'], 
    'k', 
    ['hit_rate', 'mean_avg_prec', 'ndcg'], 
    'name', 
    title="Collaborative Recommender vs Hybrid Recommender", 
    labels={"k": "k", "hit_rate": "Hit Rate", "name": "Method"},
    subplot_titles=['HitRate@k', 'MAP@k', 'NDCG@k'],
)

C:\Users\brown\AppData\Local\Temp\ipykernel_17396\4039316079.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Find Cases Where Hyrbid Outperforms Model-Based

In [206]:
full_casc_results_df = pd.read_csv("switching_hybrid_model_full_results.csv")
top_5_casc_rec_df = full_casc_results_df[full_casc_results_df['k']==5]

full_collab_results_df = pd.read_csv("collaborative_offline_eval_results_full.csv")
top_5_collab_rec_df = full_collab_results_df[full_collab_results_df['k']==5]

combined_recs_df = top_5_casc_rec_df.merge(top_5_collab_rec_df, on='userid', how='inner', suffixes=["_casc", "_collab"])
temp_df = combined_recs_df[combined_recs_df['NDCG@k_casc']>combined_recs_df['NDCG@k_collab']]

user = temp_df['userid'].values[0]

casc_df = temp_df[['k_casc', 'precision@k_casc', 'recall@k_casc', 'NDCG@k_casc']]
casc_df.columns = ['k', 'precision@5', 'recall@5', 'NDCG@5']
casc_df['name'] = "Switch Hybrid Model"

collab_df = temp_df[['k_collab', 'precision@k_collab', 'recall@k_collab', 'NDCG@k_collab']]
collab_df.columns = ['k', 'precision@5', 'recall@5', 'NDCG@5']
collab_df['name'] = "Collaborative Filtering Model"

graph_df = pd.concat([collab_df, casc_df])

create_bar_chart_plotly(
    graph_df,
    'k', 
    ['precision@5', 'recall@5', 'NDCG@5'], 
    'name', 
    title=f"Recommendation Results For User: {user}", 
    labels={"k": "k", "hit_rate": "Hit Rate", "name": "Method"},
    subplot_titles=['precision@5', 'recall@5', 'NDCG@5']
)


C:\Users\brown\AppData\Local\Temp\ipykernel_17396\2784845858.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\brown\AppData\Local\Temp\ipykernel_17396\2784845858.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [210]:
import json

def get_train_test(verbose=False):
    with open("../../data/offline_evaluation/train.json", "r") as f:
        train_set = json.load(f)
    
    with open("../../data/offline_evaluation/test.json", "r") as f:
        test_set = json.load(f)
    
    # remove users from train and test set
    # (removing those without a sample in train or test)
    del_keys  = []
    for key in train_set:
        user = key
        num_games_train = len(train_set[key])
        num_games_test = len(test_set[key])

        if num_games_train==0 or num_games_test==0: # ignore these users
            if verbose: print(f"Deleting user ({user}):\n\t{num_games_train} games in the train set\n\t{num_games_test} games in the test set")
            del_keys.append(key)
    
    for key in del_keys:
        del train_set[key]
        del test_set[key]

    return train_set, test_set

In [ ]:
# user
"""
Collaborative Model Results For user:
        appid                        name     score
    0  599390          Battle for Wesnoth  0.424866
    1  271590   Grand Theft Auto V Legacy  0.391422
    2  719890           Beasts of Bermuda  0.382591
    3  378120       Football Manager 2016  0.381385
    4    8930  Sid Meier's Civilization V  0.359771

    - hit: True, precision@k: 0.2, recall@k: 0.2, NDCG@K: 0.38653, MRR: 0.2

Hybrid Model Results For user:
rec_df:
        appid                         name     score
    0  599390           Battle for Wesnoth  0.424866
    1  271590    Grand Theft Auto V Legacy  0.391422
    2  719890            Beasts of Bermuda  0.382591
    3    8930  Sid Meier's Civilization® V  0.359771
    4     730             Counter-Strike 2  0.355516

    - hit: True, precision@k: 0.4, recall@k: 0.4, NDCG@K: 0.5012658, MRR: 
"""

76561197991621393

In [235]:
train, test = get_train_test()
print(f"Num Train Examples: {len(train[str(user)])}")
print(f"Num Test Examples: {len(test[str(user)])}")

Num Train Examples: 5
Num Test Examples: 5


In [217]:
temp_game_df = pd.read_csv("../../data/game_player_cnt_ranked_top_1k.csv")

In [231]:
sorted_train_games = sorted([(i, train[str(user)][i]) for i in train[str(user)]], key=lambda x: x[1], reverse=True)
temp_game_df.set_index("appid").loc[[int(i[0]) for i in sorted_train_games]]

,name,player_count,result
appid,,,
71270,Football Manager 2012,79.0,1
3910,Sid Meier's Civilization III: Complete,1253.0,1
230290,Universe Sandbox,367.0,1
24780,SimCity 4 Deluxe,344.0,1
578080,PUBG: BATTLEGROUNDS,207046.0,1


In [232]:
sorted_test_games = sorted([(i, test[str(user)][i]) for i in test[str(user)]], key=lambda x: x[1], reverse=True)
temp_game_df.set_index("appid").loc[[int(i[0]) for i in sorted_test_games]]

,name,player_count,result
appid,,,
730,Counter-Strike 2,682378.0,1
8930,Sid Meier's Civilization V,11670.0,1
603850,Age of History II,299.0,1
289070,Sid Meier's Civilization VI,25747.0,1
35140,Batman: Arkham Asylum GOTY Edition,681.0,1


In [239]:
top_1000_games_df = pd.read_csv("../../data/game_player_cnt_ranked_top_1k.csv")
top_1000_games_df

,appid,name,player_count,result
0,730,Counter-Strike 2,682378.0,1
1,570,Dota 2,284438.0,1
2,578080,PUBG: BATTLEGROUNDS,207046.0,1
3,252490,Rust,100213.0,1
4,359550,Tom Clancy's Rainbow Six® Siege X,82809.0,1
...,...,...,...,...
995,315660,Trainz: A New Era,67.0,1
996,441550,STAR WARS™ Rebellion,67.0,1
997,302510,Ryse: Son of Rome,67.0,1
998,423580,Project Highrise,67.0,1


In [249]:
top_1000_details_df = pd.read_csv("../../data/top_1000_game_details.csv")
top_1000_details_df

top_1000_details_df[top_1000_details_df['name'].str.lower().str.startswith("left")]

,appid,name,about_the_game,achievements,background,background_raw,capsule_image,capsule_imagev5,categories,content_descriptors,...,recommendations,release_date,required_age,reviews,screenshots,short_description,support_info,supported_languages,type,website
17,550,Left 4 Dead 2,"Set in the zombie apocalypse, Left 4 Dead 2 (L...","{'total': 101, 'highlighted': [{'name': 'CL0WN...",https://store.akamai.steamstatic.com/images/st...,https://store.akamai.steamstatic.com/images/st...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,"[{'id': 2, 'description': 'Single-player'}, {'...","{'ids': [2, 5], 'notes': 'Left 4 Dead 2 featur...",...,{'total': 738644},"{'coming_soon': False, 'date': 'Nov 16, 2009'}",0.0,NaN,"[{'id': 0, 'path_thumbnail': 'https://shared.a...","Set in the zombie apocalypse, Left 4 Dead 2 (L...","{'url': 'http://steamcommunity.com/app/550', '...","Danish, Dutch, English<strong>*</strong>, Finn...",game,http://www.l4d.com
226,500,Left 4 Dead,"From Valve (the creators of Counter-Strike, Ha...","{'total': 73, 'highlighted': [{'name': 'DEAD S...",https://store.akamai.steamstatic.com/images/st...,https://store.akamai.steamstatic.com/images/st...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,"[{'id': 2, 'description': 'Single-player'}, {'...","{'ids': [2, 5], 'notes': 'Includes realistic v...",...,{'total': 52735},"{'coming_soon': False, 'date': 'Nov 17, 2008'}",0.0,NaN,"[{'id': 0, 'path_thumbnail': 'https://shared.a...","From Valve (the creators of Counter-Strike, Ha...","{'url': 'http://steamcommunity.com/app/500', '...","English<strong>*</strong>, French<strong>*</st...",game,http://www.l4d.com/
424,564,Left 4 Dead 2 Add-on Support,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [244]:
new_game_details = top_1000_games_df[['appid', 'name']].merge(
    top_1000_details_df[[i for i in top_1000_details_df.columns if i != 'name']], 
    on="appid", 
    how="left"
).drop_duplicates(subset=['appid'])
new_game_details

,appid,name,about_the_game,achievements,background,background_raw,capsule_image,capsule_imagev5,categories,content_descriptors,...,recommendations,release_date,required_age,reviews,screenshots,short_description,support_info,supported_languages,type,website
0,730,Counter-Strike 2,"For over two decades, Counter-Strike has offer...","{'total': 1, 'highlighted': [{'name': 'A New B...",https://store.akamai.steamstatic.com/images/st...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","{'ids': [2, 5], 'notes': 'Includes intense vio...",...,{'total': 4551811},"{'coming_soon': False, 'date': '21 Aug, 2012'}",0.0,NaN,"[{'id': 0, 'path_thumbnail': 'https://shared.a...","For over two decades, Counter-Strike has offer...","{'url': '', 'email': ''}","Czech, Danish, Dutch, English<strong>*</strong...",game,http://counter-strike.net/
1,570,Dota 2,<strong>The most-played game on Steam.</strong...,NaN,https://store.akamai.steamstatic.com/images/st...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","{'ids': [5], 'notes': 'Dota 2 includes fantasy...",...,{'total': 14350},"{'coming_soon': False, 'date': '9 Jul, 2013'}",0.0,“A modern multiplayer masterpiece.”<br>9.5/10 ...,"[{'id': 0, 'path_thumbnail': 'https://shared.a...","Every day, millions of players worldwide enter...","{'url': '', 'email': ''}","Bulgarian, Czech, Danish, Dutch, English<stron...",game,http://www.dota2.com/
2,578080,PUBG: BATTLEGROUNDS,"<p class=""bb_paragraph""><img class=""bb_img"" sr...","{'total': 37, 'highlighted': [{'name': 'Last S...",https://store.akamai.steamstatic.com/images/st...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","{'ids': [2, 5], 'notes': None}",...,{'total': 1741534},"{'coming_soon': False, 'date': 'Dec 21, 2017'}",0.0,NaN,"[{'id': 0, 'path_thumbnail': 'https://shared.a...","PUBG: BATTLEGROUNDS, the high-stakes winner-ta...","{'url': 'https://support.pubg.com/hc/en-us', '...","English, Korean, Simplified Chinese, French, G...",game,https://www.pubg.com
3,252490,Rust,"<p class=""bb_paragraph"">The only aim in Rust i...","{'total': 92, 'highlighted': [{'name': 'Place ...",https://store.akamai.steamstatic.com/images/st...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,"[{'id': 1, 'description': 'Multi-player'}, {'i...","{'ids': [1, 2, 5], 'notes': 'Contains violence...",...,{'total': 1023708},"{'coming_soon': False, 'date': '8 Feb, 2018'}",0.0,"“Rust is one of the cruelest games on Steam, a...","[{'id': 0, 'path_thumbnail': 'https://shared.a...",The only aim in Rust is to survive. Everything...,{'url': 'http://support.facepunchstudios.com/'...,"English<strong>*</strong>, French<strong>*</st...",game,http://rust.facepunch.com/
4,359550,Tom Clancy's Rainbow Six® Siege X,Tom Clancy's Rainbow Six Siege X is the undisp...,NaN,https://store.akamai.steamstatic.com/images/st...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,"[{'id': 2, 'description': 'Single-player'}, {'...","{'ids': [5], 'notes': 'This is a first-person ...",...,{'total': 1199549},"{'coming_soon': False, 'date': 'Dec 1, 2015'}",17.0,NaN,"[{'id': 0, 'path_thumbnail': 'https://shared.a...",Rainbow Six Siege X is the reference in tactic...,"{'url': 'http://support.ubi.com', 'email': ''}","English<strong>*</strong>, French<strong>*</st...",game,http://rainbow6.ubi.com/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [245]:
new_game_details.to_csv("../../data/top_1000_game_details.csv", index=False)

In [248]:
new_game_details[new_game_details['name'].str.lower().str.startswith("left")]

,appid,name,about_the_game,achievements,background,background_raw,capsule_image,capsule_imagev5,categories,content_descriptors,...,recommendations,release_date,required_age,reviews,screenshots,short_description,support_info,supported_languages,type,website
17,550,Left 4 Dead 2,"Set in the zombie apocalypse, Left 4 Dead 2 (L...","{'total': 101, 'highlighted': [{'name': 'CL0WN...",https://store.akamai.steamstatic.com/images/st...,https://store.akamai.steamstatic.com/images/st...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,"[{'id': 2, 'description': 'Single-player'}, {'...","{'ids': [2, 5], 'notes': 'Left 4 Dead 2 featur...",...,{'total': 738644},"{'coming_soon': False, 'date': 'Nov 16, 2009'}",0.0,NaN,"[{'id': 0, 'path_thumbnail': 'https://shared.a...","Set in the zombie apocalypse, Left 4 Dead 2 (L...","{'url': 'http://steamcommunity.com/app/550', '...","Danish, Dutch, English<strong>*</strong>, Finn...",game,http://www.l4d.com
226,500,Left 4 Dead,"From Valve (the creators of Counter-Strike, Ha...","{'total': 73, 'highlighted': [{'name': 'DEAD S...",https://store.akamai.steamstatic.com/images/st...,https://store.akamai.steamstatic.com/images/st...,https://shared.akamai.steamstatic.com/store_it...,https://shared.akamai.steamstatic.com/store_it...,"[{'id': 2, 'description': 'Single-player'}, {'...","{'ids': [2, 5], 'notes': 'Includes realistic v...",...,{'total': 52735},"{'coming_soon': False, 'date': 'Nov 17, 2008'}",0.0,NaN,"[{'id': 0, 'path_thumbnail': 'https://shared.a...","From Valve (the creators of Counter-Strike, Ha...","{'url': 'http://steamcommunity.com/app/500', '...","English<strong>*</strong>, French<strong>*</st...",game,http://www.l4d.com/
425,564,Left 4 Dead 2 Add-on Support,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
